In [1]:
import warnings
import numpy as np
import pandas as pd
from datetime import date, datetime
from IPython.core.display import display, HTML
warnings.filterwarnings('ignore')
pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)
display(HTML('<style>.container{width : 100% ! important;}</style>'))

In [2]:
matches = pd.read_csv('Matches.csv')
balls = pd.read_csv('Match Details.csv')
print('Datasets Loaded.')

Datasets Loaded.


In [3]:
#merging matches and balls to make ipl dataset

ipl = balls.merge(matches, on = 'match_id', how = 'left')
ipl['year'] = ipl['date'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').year)
ipl['date'] = pd.to_datetime(ipl['date'])
ipl = ipl.sort_values(by = ['match_id', 'innings', 'overs', 'balls'])
ipl.reset_index(inplace = True, drop = True)
print('IPL dataset formed.')

#final score

final_score = pd.DataFrame(ipl.groupby(['match_id', 'innings'])['total_runs'].sum())
final_score.reset_index(inplace = True)
final_score.rename(columns = {'total_runs' : 'final_score'}, inplace = True)
ipl = ipl.merge(final_score, on = ['match_id', 'innings'], how = 'left')
print('final_score column completed.')

#total wickets

total_wickets = pd.DataFrame(ipl.groupby(['match_id', 'innings'])['is_wicket'].sum())
total_wickets.reset_index(inplace = True)
total_wickets.rename(columns = {'is_wicket' : 'total_wickets'}, inplace = True)
ipl = ipl.merge(total_wickets, on = ['match_id', 'innings'], how = 'left')
print('total_wickets column completed.')

#runs

ipl['runs'] = 0
ipl['runs'][0] = ipl['total_runs'][0]
ipl['runs'][ipl.shape[0]-1] = ipl['final_score'][ipl.shape[0]-1]
for i in range(1, ipl.shape[0]-1):
    if ipl['match_id'][i] == ipl['match_id'][i+1]:
        if ipl['innings'][i] == ipl['innings'][i+1]:
            ipl['runs'][i] = ipl['total_runs'][i] + ipl['runs'][i-1]
for i in range(1, ipl.shape[0]-1):
    if ipl['match_id'][i] != ipl['match_id'][i+1]:
        ipl['runs'][i] = ipl['total_runs'][i] + ipl['runs'][i-1]
for i in range(1, ipl.shape[0]-1):
    if ipl['match_id'][i] == ipl['match_id'][i+1]:
        if ipl['innings'][i] != ipl['innings'][i+1]:
            ipl['runs'][i] = ipl['total_runs'][i] + ipl['runs'][i-1]
print('runs column completed.')
            
#wickets

ipl['wickets'] = 0
ipl['wickets'][0] = ipl['is_wicket'][0]
ipl['wickets'][ipl.shape[0]-1] = ipl['total_wickets'][ipl.shape[0]-1]
for i in range(1, ipl.shape[0]-1):
    if ipl['match_id'][i] == ipl['match_id'][i+1]:
        if ipl['innings'][i] == ipl['innings'][i+1]:
            ipl['wickets'][i] = ipl['is_wicket'][i] + ipl['wickets'][i-1]
for i in range(ipl.shape[0]-1):
    if ipl['match_id'][i] != ipl['match_id'][i+1]:
        ipl['wickets'][i] = ipl['total_wickets'][i]
for i in range(ipl.shape[0]-1):
    if ipl['match_id'][i] == ipl['match_id'][i+1]:
        if ipl['innings'][i] != ipl['innings'][i+1]:
            ipl['wickets'][i] = ipl['total_wickets'][i]
print('wickets column completed.')
            
#runs last 5 overs

ipl['runs_last_5_overs'] = 0
for i in ipl.index:
    if ipl['overs'][i] <= 4:
        ipl['runs_last_5_overs'][i] = ipl['runs'][i]
for i in ipl.index:
    if ipl['overs'][i] > 4:
        if ipl['match_id'][i] == ipl['match_id'][i-30]:
            if ipl['innings'][i] == ipl['innings'][i-30]:
                ipl['runs_last_5_overs'][i] = ipl['runs'][i] - ipl['runs'][i-30]
print('runs_last_5_overs column completed.')
                
#wickets last 5 overs

ipl['wickets_last_5_overs'] = 0
for i in ipl.index:
    if ipl['overs'][i] <= 4:
        ipl['wickets_last_5_overs'][i] = ipl['wickets'][i]
for i in ipl.index:
    if ipl['overs'][i] > 4:
        if ipl['match_id'][i] == ipl['match_id'][i-30]:
            if ipl['innings'][i] == ipl['innings'][i-30]:
                ipl['wickets_last_5_overs'][i] = ipl['wickets'][i] - ipl['wickets'][i-30]
for i in ipl.index:
    if ipl['overs'][i] > 4:
        if ipl['match_id'][i] != ipl['match_id'][i-1]:
            ipl['wickets_last_5_overs'][i-1] = ipl['wickets'][i-1] - ipl['wickets'][i-31]
for i in ipl.index:
    if ipl['overs'][i] > 4:
        if ipl['match_id'][i] == ipl['match_id'][i-1]:
            if ipl['innings'][i] != ipl['innings'][i-1]:
                ipl['wickets_last_5_overs'][i-1] = ipl['wickets'][i-1] - ipl['wickets'][i-31]
print('wickets_last_5_overs column completed.')

#balls number, balls and overs

ipl['ball_no'] = 0
ipl['extras_type'] = ipl['extras_type'].astype(str)
for i in ipl.index:
    if (ipl['overs'][i] == 0) & (ipl['balls'][i] == 1):
        ipl['ball_no'][i] = 1
for i in range(1, ipl.shape[0]):
    if (ipl['overs'][i] == 0) & (ipl['balls'][i] == 1):
        pass
    else:
        if 'wides' in ipl['extras_type'][i-1]:
            ipl['ball_no'][i] = ipl['ball_no'][i-1]
        elif 'noballs' in ipl['extras_type'][i-1]:
            ipl['ball_no'][i] = ipl['ball_no'][i-1]
        else:
            ipl['ball_no'][i] = ipl['ball_no'][i-1] + 1
print('ball number completed.')

ipl['balls'] = ipl['ball_no'] % 6
print('balls completed.')

for i in ipl.index:
    if ipl['balls'][i] == 0:
        ipl['overs'][i] += 1
print('overs completed.')

#balls remaining and runs scored/required

max_balls = pd.DataFrame(ipl.groupby(['match_id', 'innings'])['ball_no'].max())
max_balls.rename(columns = {'ball_no' : 'max_balls'}, inplace = True)
max_balls.reset_index(inplace = True)
ipl = ipl.merge(max_balls, on = ['match_id', 'innings'], how = 'left')
ipl['balls_remaining'] = 0
for i in ipl.index:
    if ipl['innings'][i] == 1:
        ipl['balls_remaining'][i] = ipl['max_balls'][i] - ipl['ball_no'][i]
    else:
        ipl['balls_remaining'][i] = (ipl['target_overs'][i] * 6) - ipl['ball_no'][i]
print('balls remaining completed.')

ipl['runs_scored_in_remaining_balls'] = 0
ipl['runs_required'] = 0
for i in ipl.index:
    if ipl['innings'][i] == 1:
        ipl['runs_scored_in_remaining_balls'][i] = ipl['final_score'][i] - ipl['runs'][i]
    else:
        ipl['runs_required'][i] = ipl['target_runs'][i] - ipl['runs'][i]
print('runs scored/required completed.')

#won

won = []
for i in ipl.index:
    if ipl['batting_team'][i] == ipl['winner'][i]:
        won.append(1)
    else:
        won.append(0)
ipl['won'] = won
print('won column created.')

#cleaning teams

for col in ['team1', 'team2', 'batting_team', 'bowling_team']:
    t_list = []
    for i in ipl[col]:
        if i == 'Delhi Daredevils':
            t_list.append('Delhi Capitals')
        elif i == 'Rising Pune Supergiants':
            t_list.append('Rising Pune Supergiant')
        elif i == 'Kings XI Punjab':
            t_list.append('Punjab Kings')
        else:
            t_list.append(i)
    ipl[col] = t_list
print(col + ' cleaned.')

#cleaning venues

t_list = []
for i in ipl['venue']:
    if i == 'M.Chinnaswamy Stadium':
        t_list.append('M Chinnaswamy Stadium')
    elif i == 'MA Chidambaram Stadium, Chepauk, Chennai':
        t_list.append('MA Chidambaram Stadium')
    elif i == 'MA Chidambaram Stadium, Chepauk':
        t_list.append('MA Chidambaram Stadium')
    elif i == 'Subrata Roy Sahara Stadium':
        t_list.append('Maharashtra Cricket Association Stadium')
    elif i == 'Sardar Patel Stadium, Motera':
        t_list.append('Narendra Modi Stadium')
    elif i == 'Narendra Modi Stadium, Ahmedabad':
        t_list.append('Narendra Modi Stadium')
    elif i == 'Wankhede Stadium, Mumbai':
        t_list.append('Wankhede Stadium')
    elif i == 'Feroz Shah Kotla':
        t_list.append('Arun Jaitley Stadium')
    elif i == 'Arun Jaitley Stadium, Delhi':
        t_list.append('Arun Jaitley Stadium')
    elif i == 'Brabourne Stadium, Mumbai':
        t_list.append('Brabourne Stadium')
    elif i == 'Punjab Cricket Association IS Bindra Stadium, Mohali':
        t_list.append('Punjab Cricket Association IS Bindra Stadium')
    elif i == 'Punjab Cricket Association Stadium, Mohali':
        t_list.append('Punjab Cricket Association IS Bindra Stadium')
    elif i == 'Rajiv Gandhi International Stadium, Uppal':
        t_list.append('Rajiv Gandhi International Stadium')
    elif i == 'Zayed Cricket Stadium, Abu Dhabi':
        t_list.append('Sheikh Zayed Stadium')
    elif i == 'Vidarbha Cricket Association Stadium, Jamtha':
        t_list.append('Vidarbha Cricket Association Stadium')
    else:
        t_list.append(i)
ipl['venue'] = t_list
print('venue cleaned.')

IPL dataset formed.
final_score column completed.
total_wickets column completed.
runs column completed.
wickets column completed.
runs_last_5_overs column completed.
wickets_last_5_overs column completed.
ball number completed.
balls completed.
overs completed.
balls remaining completed.
runs scored/required completed.
won column created.
bowling_team cleaned.
venue cleaned.


In [4]:
ipl.columns

Index(['match_id', 'innings', 'overs', 'balls', 'batsman', 'non_striker',
       'bowler', 'batsman_runs', 'extra_runs', 'total_runs', 'non_boundary',
       'is_wicket', 'dismissal_kind', 'player_dismissed', 'fielder',
       'extras_type', 'batting_team', 'bowling_team', 'target_overs',
       'target_runs', 'year', 'city', 'date', 'player_of_match', 'venue',
       'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'result',
       'result_margin', 'method', 'players', 'umpire1', 'umpire2', 'playoff',
       'knockout', 'final', 'final_score', 'total_wickets', 'runs', 'wickets',
       'runs_last_5_overs', 'wickets_last_5_overs', 'ball_no', 'max_balls',
       'balls_remaining', 'runs_scored_in_remaining_balls', 'runs_required',
       'won'],
      dtype='object')

In [5]:
#rearranging columns

ipl = ipl[['match_id', 'year', 'date', 'city', 'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'innings', 'batting_team', 
           'bowling_team', 'batsman', 'non_striker', 'bowler', 'overs', 'balls', 'batsman_runs', 'extra_runs', 'total_runs', 
           'non_boundary', 'is_wicket', 'dismissal_kind', 'player_dismissed', 'fielder', 'extras_type', 'runs', 'wickets', 'runs_last_5_overs', 
           'wickets_last_5_overs', 'target_overs', 'target_runs', 'balls_remaining', 'runs_scored_in_remaining_balls', 'runs_required', 
           'winner', 'result', 'result_margin', 'player_of_match', 'playoff', 'knockout', 'final', 'method', 'players', 'umpire1', 'umpire2', 
           'final_score', 'total_wickets']]

In [6]:
#1st and 2nd Innings data

ipl_1 = ipl[ipl['innings'] == 1]
ipl_2 = ipl[ipl['innings'] == 2]

In [7]:
#merging scores and wickets to match list

matches = matches.merge(ipl_1[['match_id', 'final_score', 'total_wickets']], on = 'match_id', how = 'left')
matches.drop_duplicates(inplace = True)
matches.rename(columns = {'final_score' : 'innings_1_score', 'total_wickets' : 'innings_1_wickets'}, inplace = True)
matches = matches.merge(ipl_2[['match_id', 'final_score', 'total_wickets']], on = 'match_id', how = 'left')
matches.drop_duplicates(inplace = True)
matches.rename(columns = {'final_score' : 'innings_2_score', 'total_wickets' : 'innings_2_wickets'}, inplace = True)

In [8]:
#batting and bowling teams

innings_1_bat_team, innings_2_bat_team = [], []
for i in matches.index:
    if (matches['team1'][i] == matches['toss_winner'][i]) and (matches['toss_decision'][i] == 'bat'):
        innings_1_bat_team.append(matches['team1'][i])
        innings_2_bat_team.append(matches['team2'][i])
    elif (matches['team1'][i] == matches['toss_winner'][i]) and (matches['toss_decision'][i] == 'field'):
        innings_1_bat_team.append(matches['team2'][i])
        innings_2_bat_team.append(matches['team1'][i])
    elif (matches['team2'][i] == matches['toss_winner'][i]) and (matches['toss_decision'][i] == 'bat'):
        innings_1_bat_team.append(matches['team2'][i])
        innings_2_bat_team.append(matches['team1'][i])
    elif (matches['team2'][i] == matches['toss_winner'][i]) and (matches['toss_decision'][i] == 'field'):
        innings_1_bat_team.append(matches['team1'][i])
        innings_2_bat_team.append(matches['team2'][i])
    else:
        innings_1_bat_team.append(np.nan)
        innings_2_bat_team.append(np.nan)
matches['innings_1_bat_team'] = innings_1_bat_team
matches['innings_2_bat_team'] = innings_2_bat_team

In [9]:
#Saving files

ipl.to_csv('IPL.csv', index = None)
ipl_1.to_csv('IPL 1st Inningsv2.csv', index = None)
ipl_2.to_csv('IPL 2nd Inningsv2.csv', index = None)
matches.to_csv('Match Processed.csv', index = None)